In [3]:
from IPython.display import display, HTML
display(HTML ("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input{font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. tensorflow v2.xx에서 v1 사용하기

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() # tensorflow V2 비활성화하고 V1 활성화
import numpy as np
import pandas as pd

## Tensorflow
- 데이터 흐름 그래프(tensor 객체의 흐름)을 사용하는 수치 계산 라이브러리
- 그래프는 node(연산)와 edge로 구성
- sess = tf.Session()을 이용해서 실행 환경
- sess.run()을 통해서 실행 결과를 확인

In [8]:
# 1단계 : tensor 정의
node1 = tf.constant('Hello, Tensorflow')
# 2단계 : 세션
sess = tf.Session()
# 3단계 : 실제 실행
print(sess.run(node1))
print(sess.run(node1).decode())

b'Hello, Tensorflow'
Hello, Tensorflow


In [12]:
# 간단한 연산 tensor 그래프
# 1. 그래프 정의
node1 = tf.constant(10, dtype=tf.float16)
node2 = tf.constant(20, dtype=tf.float16)
node3 = tf.add(node1, node2)
# 2단계 : 세션 생성
sess = tf.Session()
# 3. 세션 실행 & 결과
print(sess.run([node1, node2, node3]))

[10.0, 20.0, 30.0]


In [13]:
# 타입변경
import numpy as np
node1 = tf.constant(np.array([1,2,3]), dtype=tf.int16)
node2 = tf.cast(node1, dtype=tf.float32)
sess = tf.Session()
print(sess.run([node1, node2]))

[array([1, 2, 3], dtype=int16), array([1., 2., 3.], dtype=float32)]


In [15]:
# 평균값 계산 : tf.reduce_mean()
data = np.array([1.,2.,3.,4.])
m = tf.reduce_mean(data)
sess = tf.Session()
print(sess.run(m))

2.5


In [16]:
# tf.random_normal([size]) : 평균이 0이고 표준편차가 1인 난수 size개 발생
w = tf.random_normal([5])
sess = tf.Session()
sess.run(w)

array([-0.16933092, -0.61927193,  0.5688556 , -1.9350089 ,  1.3643692 ],
      dtype=float32)

In [17]:
# 변수 노드
w = tf.Variable(tf.random_normal([1]))
sess = tf.Session()
sess.run(tf.global_variables_initializer()) #변수 초기화
sess.run(w)

array([-1.7817266], dtype=float32)

# 2. tensorflow v1을 이용한 회귀분석 구현
## 2.1 독립변수 x가 1개, 타겟(종속)변수 y가 1개

In [18]:
# tensor 그래프 정의
# 데이터 셋 확보
x = np.array([1,2,3])
y = np.array([2,3,4])
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = W * x + b
# cost function (손실함수 : mse )
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프. 곡선위 미분값이 0이 되는 방향
(경사하강법GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2 에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b])
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))

0번째 cost:0.020911777392029762, W:[0.901036], b:[1.0959309]
300번째 cost:0.000464440236100927, W:[0.97503024], b:[1.0567622]
600번째 cost:0.00010959017527056858, W:[0.9878707], b:[1.0275728]
900번째 cost:2.5854722480289638e-05, W:[0.99410856], b:[1.0133926]
1200번째 cost:6.101076451159315e-06, W:[0.99713814], b:[1.0065057]
1500번째 cost:1.4397816130440333e-06, W:[0.99860966], b:[1.0031605]
1800번째 cost:3.4002732718363404e-07, W:[0.9993244], b:[1.0015358]
2100번째 cost:8.04068989168627e-08, W:[0.9996714], b:[1.000747]
2400번째 cost:1.911402591758815e-08, W:[0.99983984], b:[1.0003641]
2700번째 cost:4.6101007455945364e-09, W:[0.99992114], b:[1.0001788]
3000번째 cost:1.1454043269409908e-09, W:[0.9999609], b:[1.000089]
3300번째 cost:3.061112752167361e-10, W:[0.99997985], b:[1.0000463]
3600번째 cost:8.733517381509515e-11, W:[0.9999891], b:[1.0000248]
3900번째 cost:8.733517381509515e-11, W:[0.9999891], b:[1.0000248]
4200번째 cost:8.733517381509515e-11, W:[0.9999891], b:[1.0000248]
4500번째 cost:8.733517381509515e-11, W:[0

In [19]:
# 최종적으로 나온 회귀식 H = W*x + b
W_, b_ = sess.run([W, b])

In [20]:
W_, b_

(array([0.9999891], dtype=float32), array([1.0000248], dtype=float32))

In [21]:
def predict(x):
    return W_[0] * x + b_[0]

In [22]:
predict(5)

5.999970257282257

## 2.2 predict을 위한 placeholder 이용
- placeholder : 외부에서 데이터를 입력받을 수 있는 노드

In [23]:
x = tf.placeholder(dtype=tf.float32)
H = 1*x + 1
sess = tf.Session()
sess.run([x, H], feed_dict={x : np.array([40,50])})

[array([40., 50.], dtype=float32), array([41., 51.], dtype=float32)]

In [24]:
# tensor 그래프 정의
# 데이터 셋 확보
x_data = np.array([1,2,3])
y_data = np.array([2,3,4])
# placeholder 설정(x, y)
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis : 결과는 numpy 배열
H = W * x + b
# cost function (손실함수 : mse )
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프. 곡선위 미분값이 0이 되는 방향
(경사하강법GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습 (v2 에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b],
                                        feed_dict={x:x_data,
                                                   y:y_data})
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))

0번째 cost:20.970300674438477, W:[-0.24866721], b:[-0.4464114]
300번째 cost:0.01921742968261242, W:[1.1606195], b:[0.6348743]
600번째 cost:0.004534522537142038, W:[1.0780219], b:[0.82263786]
900번째 cost:0.0010699575068429112, W:[1.0378994], b:[0.91384554]
1200번째 cost:0.00025246257428079844, W:[1.0184097], b:[0.9581501]
1500번째 cost:5.95706223975867e-05, W:[1.0089426], b:[0.9796711]
1800번째 cost:1.405638613505289e-05, W:[1.004344], b:[0.99012506]
2100번째 cost:3.3172811981785344e-06, W:[1.0021102], b:[0.99520284]
2400번째 cost:7.832341566427203e-07, W:[1.0010256], b:[0.9976688]
2700번째 cost:1.853204594226554e-07, W:[1.000499], b:[0.9988663]
3000번째 cost:4.402039621709264e-08, W:[1.0002433], b:[0.99944746]
3300번째 cost:1.0472017031304404e-08, W:[1.0001186], b:[0.9997305]
3600번째 cost:2.54033238888951e-09, W:[1.0000585], b:[0.99986756]
3900번째 cost:5.904941535739283e-10, W:[1.0000281], b:[0.9999359]
4200번째 cost:1.631974555493798e-10, W:[1.000015], b:[0.9999665]
4500번째 cost:5.195962063386794e-11, W:[1.00000

In [26]:
# 예측하기
sess.run(H, feed_dict={x:5})

array([50.568027], dtype=float32)

# 2.3 scale이 다른 데이터들의 회귀분석 구현(scale 조정X)

In [29]:
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight & bias
W = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))
# Hypothesis
H = W * x + b
# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cost)
# Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 6001):
    _, cost_val= sess.run([train, cost], feed_dict={x:x_data, y:y_data})
    if step%300==0:
        print("{}/6000번째 cost:{}".format(step+1, cost_val))

301/6000번째 cost:124.55949401855469
601/6000번째 cost:80.52308654785156
901/6000번째 cost:80.11064910888672
1201/6000번째 cost:80.07347106933594
1501/6000번째 cost:80.0406494140625
1801/6000번째 cost:80.0090103149414
2101/6000번째 cost:79.9785385131836
2401/6000번째 cost:79.94908142089844
2701/6000번째 cost:79.92064666748047
3001/6000번째 cost:79.89324188232422
3301/6000번째 cost:79.86682891845703
3601/6000번째 cost:79.84130859375
3901/6000번째 cost:79.81665802001953
4201/6000번째 cost:79.79291534423828
4501/6000번째 cost:79.7699966430664
4801/6000번째 cost:79.74787902832031
5101/6000번째 cost:79.72650909423828
5401/6000번째 cost:79.70592498779297
5701/6000번째 cost:79.68603515625
6001/6000번째 cost:79.66687774658203


## 2.4 scale이 다른 데이터들의 회귀분석 구현(scale 조정O)
### scale 조정 방법: 모든 데이터를 일정범위내로 조정
- normalization(정규화) : 모든 데이터를 0~1 사이로 조정
                            X - Xmin
    normalization = ────────────────────────
                            Xmax -Xmin
       * 위의 식을 써도 되지만 라이브러리(sklearn.preprocessing.MinMaxScaler)를 주로 씀
- standarization(표준화) : 데이터의 평균을 0, 표준편차를 1로 조정
                           X - Xmean
    standarization = ───────────────────
                           Xstd(표준편차)
        * 위의 식보다 라이브러리 추천(sklearn.preproessing.StandardScaler)

In [38]:
# 라이브러리 쓰지 않고 정규화
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])
norm_scaled_x_data = (x_data - x_data.min()) / (x_data.max() - x_data.min())
norm_scaled_y_data = (y_data - y_data.min()) / (y_data.max() - y_data.min())

array([0.        , 0.11111111, 0.7       , 0.83333333, 1.        ])

In [44]:
# 라이브러리 사용하여 정규화
x_data = np.array([1,2,5,8,10]).reshape(-1,1)
y_data = np.array([5,15,68,80,95]).reshape(-1,1)
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler() # 독립변수 x를 정규화 시킬 객체
norm_scaled_x_data = scaler_x.fit_transform(x_data)
scaler_y = MinMaxScaler() # 종속(타겟)변수 y를 정규화시킬 객체
norm_scaled_y_data = scaler_y.fit_transform(y_data)
norm_scaled_y_data

array([[0.        ],
       [0.11111111],
       [0.7       ],
       [0.83333333],
       [1.        ]])

In [46]:
# 라이브러리 쓰지 않고 표준화
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])
stan_scaled_x_data = (x_data - x_data.mean()) / x_data.std()
stan_scaled_y_data = (y_data - y_data.mean()) / y_data.std()
np.column_stack([x_data, norm_scaled_x_data, stan_scaled_x_data])

array([[ 1.        ,  0.        , -1.22474487],
       [ 2.        ,  0.11111111, -0.93313895],
       [ 5.        ,  0.44444444, -0.05832118],
       [ 8.        ,  0.77777778,  0.81649658],
       [10.        ,  1.        ,  1.39970842]])

In [47]:
np.column_stack([y_data, norm_scaled_y_data, stan_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])

In [51]:
# 라이브러리 사용하여 표준화
x_data = np.array([1,2,5,8,10]).reshape(-1,1)
y_data = np.array([5,15,68,80,95]).reshape(-1,1)

from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
stan_scaled_x_data = scaler_x.fit_transform(x_data)
scaler_y = StandardScaler()
stan_scaled_y_data = scaler_y.fit_transform(y_data)

In [52]:
# 스케일 조정된 데이터를 다시 복규 : inverse_transform() 이용
scaler_y.inverse_transform(stan_scaled_y_data)

array([[ 5.],
       [15.],
       [68.],
       [80.],
       [95.]])

In [59]:
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight & bias
W = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))
# Hypothesis
H = W * x + b
# cost (손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 6001):
    _, cost_val= sess.run([train, cost], feed_dict={x:norm_scaled_x_data, 
                                                    y:norm_scaled_y_data})
    if step%300==0:
        print("{}/6000번째 cost:{}".format(step+1, cost_val))

301/6000번째 cost:0.039240092039108276
601/6000번째 cost:0.01704229973256588
901/6000번째 cost:0.011564750224351883
1201/6000번째 cost:0.010213102214038372
1501/6000번째 cost:0.00987956952303648
1801/6000번째 cost:0.009797262027859688
2101/6000번째 cost:0.009776952676475048
2401/6000번째 cost:0.009771943092346191
2701/6000번째 cost:0.009770706295967102
3001/6000번째 cost:0.009770401753485203
3301/6000번째 cost:0.00977032445371151
3601/6000번째 cost:0.009770305827260017
3901/6000번째 cost:0.009770299308001995
4201/6000번째 cost:0.009770301170647144
4501/6000번째 cost:0.009770301170647144
4801/6000번째 cost:0.009770302101969719
5101/6000번째 cost:0.009770302101969719
5401/6000번째 cost:0.009770302101969719
5701/6000번째 cost:0.009770302101969719
6001/6000번째 cost:0.009770302101969719
